##### Zadanie Domowe

Poniżej podany jest zestaw danych. Zawiera on brakujące wartości. Twoim zadaniem jest przygotować ten zestaw danych do obróbki a następnie wytrenować klasyfikator przewidujący choroby serca dobierając ilość kroków oraz learning rate.

Wartością docelową jest `num`

Twoim celem jest odrzucenie zmiennych charakteryzujących się dużą ilością wartości brakujących oraz uzupełnienie pozostałych kolumn. Użyj do tego na przykład funkcji `pd.DataFrame.isna`, `pd.DataFrame.sum` oraz metody `pd.Series.fillna`.

Użyj metody `xgb.cv` do ustalenia przebiegu treningu. Postaraj się dobrać takie wartości ilości kroków oraz learning rate aby uniknąć przeuczenia (overfitu) w końcowym modelu. Zacznij od learning rate `0.01`. Kiedy klasyfikator zostanie przetrenowany, użyj wykresu błędu w zależności od ilości kroków aby wyznaczyć przybliżoną optymalną ilość drzew dla danego modelu i wytrenuj ostateczny model.

Dodatkowe info: `xgb` obsługuje brakujące wartości przez parametr `missing`. Możesz (nie musisz) użyć tego zamiast interpolacji brakujących wartości.

In [78]:
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
data = pd.read_csv("heart.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         294 non-null    int64  
 1   sex         294 non-null    int64  
 2   cp          294 non-null    int64  
 3   trestbps    294 non-null    object 
 4   chol        294 non-null    object 
 5   fbs         294 non-null    object 
 6   restecg     294 non-null    object 
 7   thalach     294 non-null    object 
 8   exang       294 non-null    object 
 9   oldpeak     294 non-null    float64
 10  slope       294 non-null    object 
 11  ca          294 non-null    object 
 12  thal        294 non-null    object 
 13  num         294 non-null    int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 32.3+ KB


In [4]:
data.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num       '],
      dtype='object')

In [5]:
data.describe()

,age,sex,cp,oldpeak,num
count,294.000000,294.000000,294.000000,294.000000,294.000000
mean,47.826531,0.724490,2.982993,0.586054,0.360544
std,7.811812,0.447533,0.965117,0.908648,0.480977
min,28.000000,0.000000,1.000000,0.000000,0.000000
25%,42.000000,0.000000,2.000000,0.000000,0.000000
50%,49.000000,1.000000,3.000000,0.000000,0.000000
75%,54.000000,1.000000,4.000000,1.000000,1.000000
max,66.000000,1.000000,4.000000,5.000000,1.000000


In [6]:
data
#rekordy ze znakami zapytania które są dziwne, trzeba to wymienić na zera

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,52,1,4,160,331,0,0,94,1,2.5,?,?,?,1
290,54,0,3,130,294,0,1,100,1,0.0,2,?,?,1
291,56,1,4,155,342,1,0,150,1,3.0,2,?,?,1
292,58,0,2,180,393,0,0,110,1,1.0,2,?,7,1


In [25]:
data.isna().sum()
#brak pustych rekordów

age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
num           0
dtype: int64

In [28]:
for i in data:
     data[i] = data[i].replace('?',np.nan)

In [30]:
data.isna().sum()
#w kolumnach slope, thal i ca braki są bardzo duże,powinno się je usunąć bo i tak nie weźmieny z nich żadnych informacji, oraz nie można ich uzupełnić 

age             0
sex             0
cp              0
trestbps        1
chol           23
fbs             8
restecg         1
thalach         1
exang           1
oldpeak         0
slope         190
ca            291
thal          266
num             0
dtype: int64

In [36]:
data = data.drop(columns = ['ca', 'thal', 'slope'])

In [52]:
#jednak kolumny tahalach, exang trzeba wyrzucić bo brudzą przy One Hot encodingu
data = data.drop(columns=['thalach', 'exang'])

In [37]:
data.isna().sum()
#tutaj, resztę pustych można uzupełnić 0, bo nie ma ich aż tak dużo 


age            0
sex            0
cp             0
trestbps       1
chol          23
fbs            8
restecg        1
thalach        1
exang          1
oldpeak        0
num            0
dtype: int64

In [40]:
data = data.fillna('0')

In [41]:
data.isna().sum()

age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
num           0
dtype: int64

In [53]:
data.describe()

,age,sex,cp,oldpeak,num,num
count,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000
mean,47.826531,0.724490,2.982993,0.586054,0.360544,0.360544
std,7.811812,0.447533,0.965117,0.908648,0.480977,0.480977
min,28.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,42.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,49.000000,1.000000,3.000000,0.000000,0.000000,0.000000
75%,54.000000,1.000000,4.000000,1.000000,1.000000,1.000000
max,66.000000,1.000000,4.000000,5.000000,1.000000,1.000000


In [54]:
data.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'oldpeak',
       'num       ', 'num'],
      dtype='object')

In [55]:
#trzeba naprawić kolumne num
data['num'] = data['num       ']
data = data.drop(columns=['num       '])

In [56]:
data

,age,sex,cp,trestbps,chol,fbs,restecg,oldpeak,num
0,28,1,2,130,132,0,2,0.0,0
1,29,1,2,120,243,0,0,0.0,0
2,29,1,2,140,0,0,0,0.0,0
3,30,0,1,170,237,0,1,0.0,0
4,31,0,2,100,219,0,1,0.0,0
...,...,...,...,...,...,...,...,...,...
289,52,1,4,160,331,0,0,2.5,1
290,54,0,3,130,294,0,1,0.0,1
291,56,1,4,155,342,1,0,3.0,1
292,58,0,2,180,393,0,0,1.0,1


In [67]:
for i in ["sex", "cp", "fbs", "restecg", "oldpeak" ,"num"]:
    data[i] = data[i].astype("category")

for i in ["age", "trestbps", "chol"]:
    data[i] = data[i].astype("float64")
    
#trzeba przerobić te dane na odpowiednie rodzaję, zgodnie z specyfiką 

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       294 non-null    float64 
 1   sex       294 non-null    category
 2   cp        294 non-null    category
 3   trestbps  294 non-null    float64 
 4   chol      294 non-null    float64 
 5   fbs       294 non-null    category
 6   restecg   294 non-null    category
 7   oldpeak   294 non-null    category
 8   num       294 non-null    category
dtypes: category(6), float64(3)
memory usage: 9.8 KB


In [69]:
#XGBoost
X = data.drop(columns=["num"])
Y = data["num"]
X = pd.get_dummies(X, drop_first= True)

In [70]:
X

,age,trestbps,chol,sex_1,cp_2,cp_3,cp_4,fbs_1,restecg_1,restecg_2,oldpeak_0.5,oldpeak_0.8,oldpeak_1.0,oldpeak_1.5,oldpeak_2.0,oldpeak_2.5,oldpeak_3.0,oldpeak_4.0,oldpeak_5.0
0,28.0,130.0,132.0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,29.0,120.0,243.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,29.0,140.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,30.0,170.0,237.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,31.0,100.0,219.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,52.0,160.0,331.0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
290,54.0,130.0,294.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
291,56.0,155.0,342.0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0
292,58.0,180.0,393.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [74]:
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 10)

In [76]:
params_grid = {"max_depth": [5],"learning_rate": np.linspace(0.1,50)}

In [80]:
Classifier = xgb.XGBClassifier(objective="binary:logistic", eval_metric = "auc")
grid_search = GridSearchCV(Classifier, params_grid, cv=5, verbose = 0, n_jobs=2).fit(X_train, Y_train)
#używamy grid searcha w celu znalezienia najlepszych parametrów

C:\Users\siedl\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [87]:
grid_search.best_score_

0.7462264150943396

In [88]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)